In [4]:
import pandas as pd
import sqlite3
import os
import tkinter as tk
from tkinter import filedialog
import xml.etree.ElementTree as ET
from datetime import datetime

# Ask user to select folder containing the Session XML file
root = tk.Tk()
root.withdraw()  # Hide the root window
selected_folder = filedialog.askdirectory(initialdir='D:/Pro-Sup Test/Data/')
if not selected_folder:
    raise ValueError("No folder was selected.")

# Extract the test_date from the selected folder name
folder_name = os.path.basename(selected_folder)
test_date = folder_name.split('_', 1)[0]  # Extract '2024-08-13' from '2024-08-13_105_Growth Plate_'

# Find the XML file titled "Session" in the selected folder
xml_file_path = ''
for r, dirs, files in os.walk(selected_folder):
    for file in files:
        if file.lower().startswith('session') and file.lower().endswith('.xml'):
            xml_file_path = os.path.join(r, file)
            break
    if xml_file_path:
        break

if not xml_file_path:
    raise FileNotFoundError("No 'Session' XML file found in the selected folder.")

# Parse the XML file
tree = ET.parse(xml_file_path)
root_xml = tree.getroot()

# Extract required fields from XML
name = root_xml.find(".//Name").text
dob = root_xml.find(".//DOB").text
height = root_xml.find(".//Height").text
weight = root_xml.find(".//Weight").text
injury_history = root_xml.find(".//Injury_History").text
season_phase = root_xml.find(".//Season_Phase").text
dynomometer_score = root_xml.find(".//Dynamometer_Score_Dominant").text
comments = root_xml.find(".//Comments").text

# Calculate age from DOB
dob_date = datetime.strptime(dob, "%Y-%m-%d")
today = datetime.today()
age = today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

# Define final columns
final_columns = [
    'name', 'test_date', 'age', 'height', 'weight', 'injury_history', 'season_phase', 'dynomometer_score', 'comments',
    'forearm_rom_0to10', 'forearm_rom_10to20', 'forearm_rom_20to30', 'forearm_rom',
    'tot_rom_0to10', 'tot_rom_10to20', 'tot_rom_20to30', 'tot_rom',
    'num_of_flips_0_10', 'num_of_flips_10_20', 'num_of_flips_20_30', 'num_of_flips',
    'avg_velo_0_10', 'avg_velo_10_20', 'avg_velo_20_30', 'avg_velo'
]

# Create a DataFrame with XML data and NULL for ASCII columns
data = {
    'name': name,
    'test_date': test_date,
    'age': age,
    'height': height,
    'weight': weight,
    'injury_history': injury_history,
    'season_phase': season_phase,
    'dynomometer_score': dynomometer_score,
    'comments': comments,
    'forearm_rom_0to10': None,
    'forearm_rom_10to20': None,
    'forearm_rom_20to30': None,
    'forearm_rom': None,
    'tot_rom_0to10': None,
    'tot_rom_10to20': None,
    'tot_rom_20to30': None,
    'tot_rom': None,
    'num_of_flips_0_10': None,
    'num_of_flips_10_20': None,
    'num_of_flips_20_30': None,
    'num_of_flips': None,
    'avg_velo_0_10': None,
    'avg_velo_10_20': None,
    'avg_velo_20_30': None,
    'avg_velo': None
}

df = pd.DataFrame([data], columns=final_columns)

# Connect to the SQLite database and create table if needed
db_path = 'D:/Pro-Sup Test/pro-sup_data.sqlite'
conn = sqlite3.connect(db_path)

# Ensure table exists
df.head(0).to_sql('pro_sup_data', conn, if_exists='append', index=False)

# Insert the row with XML data and NULL ASCII columns
df.to_sql('pro_sup_data', conn, if_exists='append', index=False)

conn.close()

# In the cell that pulls the XML/ASCII
global NEW_NAME, NEW_TEST_DATE
NEW_NAME = root_xml.find(".//Name").text
NEW_TEST_DATE = folder_name.split('_', 1)[0]

print("Globals set:", NEW_NAME, NEW_TEST_DATE)

print(f"XML data inserted for {name} with test date {test_date}.")


Globals set: File, Reese 2025-10-30
XML data inserted for File, Reese with test date 2025-10-30.


In [5]:
import re
import pandas as pd
import sqlite3


def extract_test_date_from_ascii(ascii_file_path: str) -> str:
    """
    Extracts the test date in 'YYYY-MM-DD' format from the first file path in the ASCII file.
    
    Args:
        ascii_file_path (str): Path to the ASCII file.
    
    Returns:
        str: Extracted test date in 'YYYY-MM-DD' format.
    """
    with open(ascii_file_path, 'r') as file:
        lines = file.readlines()
        # Extract the first file path
        first_file_path = lines[0].strip().split('\t')[0]
        # Split the path to navigate the folder structure
        parts = first_file_path.split('\\')  # Split by folder structure
        if len(parts) > 4:  # Ensure we have enough subfolders
            date_folder = parts[4]  # Get the folder containing the date
            # Use regex to extract 'YYYY-MM-DD' format
            match = re.match(r'^\d{4}-\d{2}-\d{2}', date_folder)
            if match:
                return match.group(0)  # Return the matched date
            else:
                raise ValueError(f"Unable to extract test date from folder: {date_folder}")
        else:
            raise ValueError("Unexpected file path structure: Unable to extract test date.")


# Path to ASCII file
text_file_path = 'D:/Pro-Sup Test/pro-sup_data.txt'

# Dynamically extract the test date from the ASCII file
test_date = extract_test_date_from_ascii(text_file_path)
print(f"Extracted test date: {test_date}")

with open(text_file_path, 'r') as file:
    lines = file.readlines()
    data = [line.strip().split('\t') for line in lines]

# Extract header and data rows
header = data[1]
df_data = [row[1:] for row in data[5:]]
df_ascii = pd.DataFrame(df_data, columns=header)

# Replace hyphens with underscores
df_ascii.columns = [col.replace('-', '_') for col in df_ascii.columns]

row_ascii = df_ascii.iloc[0]

# Extract ASCII data
values = {col: row_ascii.get(col) for col in df_ascii.columns}

# Construct the UPDATE statement
set_clause = ", ".join([f"{col} = ?" for col in values.keys()])
params = list(values.values()) + [name, test_date]

update_sql = f"UPDATE pro_sup_data SET {set_clause} WHERE name = ? AND test_date = ?;"

conn = sqlite3.connect('D:/Pro-Sup Test/pro-sup_data.sqlite')
cursor = conn.cursor()
cursor.execute(update_sql, params)
conn.commit()
conn.close()

print(f"ASCII data updated for {name} on test date {test_date}.")

ascii_file_path = 'D:/Pro-Sup Test/pro-sup_data.txt'
test_date = extract_test_date_from_ascii(ascii_file_path)
print(f"Extracted test date: {test_date}")


Extracted test date: 2025-10-30
ASCII data updated for File, Reese on test date 2025-10-30.
Extracted test date: 2025-10-30


In [6]:
import sqlite3
import pandas as pd

# Path to your database
db_path = "D:/Pro-Sup Test/pro-sup_data.sqlite"

# Load the database and fetch the data
connection = sqlite3.connect(db_path)
query = "SELECT * FROM pro_sup_data;"
df = pd.read_sql_query(query, connection)

# Convert the ROM columns to numeric types for calculations
rom_columns = ['tot_rom_0to10', 'tot_rom_10to20', 'tot_rom_20to30']
df[rom_columns] = df[rom_columns].apply(pd.to_numeric, errors='coerce')

# Calculate fatigue indices for each interval
df['fatigue_index_10'] = 0  # First interval has no prior data to compare
df['fatigue_index_20'] = ((df['tot_rom_10to20'] - df['tot_rom_0to10']) / df['tot_rom_0to10']) * 100
df['fatigue_index_30'] = ((df['tot_rom_20to30'] - df['tot_rom_10to20']) / df['tot_rom_10to20']) * 100

# Calculate the total fatigue score (sum of absolute fatigue indices)
df['total_fatigue_score'] = df[['fatigue_index_10', 'fatigue_index_20', 'fatigue_index_30']].abs().sum(axis=1)

# Calculate a consistency penalty (standard deviation of ROM across intervals)
df['consistency_penalty'] = df[rom_columns].std(axis=1)

# Define max values for scaling
max_rom = df[rom_columns].max().sum()  # Max total ROM across all intervals
max_fatigue_score = df['total_fatigue_score'].max()
max_consistency_penalty = df['consistency_penalty'].max()

# Adjusted weights (giving more importance to total ROM, reducing fatigue impact)
w1, w2, w3 = 70, 15, 15  # More weight on ROM, lower penalty for fatigue

# Calculate new total score
df['total_score'] = (
    (df[rom_columns].sum(axis=1) / max_rom) * w1 - 
    (df['total_fatigue_score'] / max_fatigue_score) * w2 - 
    (df['consistency_penalty'] / max_consistency_penalty) * w3
)

# Save the updated data back to the database
with connection:
    df.to_sql('pro_sup_data', connection, if_exists='replace', index=False)

# Close the connection
connection.close()

print("Updated total score with increased ROM weighting and reduced fatigue penalty successfully added!")


Updated total score with increased ROM weighting and reduced fatigue penalty successfully added!


In [7]:
# =================  PRO‑SUP PDF REPORT  v3  (gradient, bar‑charts) ============
import os, sqlite3
import pandas as pd
import plotly.graph_objects as go
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.utils import ImageReader

# ──────────────────────────────────────────────────────────────── CONFIG ─────
DB_PATH    = r"D:/Pro-Sup Test/pro-sup_data.sqlite"
LOGO_PATH  = r"C:/Users/q/PycharmProjects/Rename_Session.xml/8ctane - Faded 8 to Blue.png"           # update path
REPORT_DIR = r"D:/Pro-Sup Test/Reports"
ACCENT_HEX = "#4887a8"       # new teal
MUTED_HEX  = "#586c7b"       # muted blue‑gray
os.makedirs(REPORT_DIR, exist_ok=True)

# ───────────────────────────────────────────────────── 1) LOAD / PREP DATA ───
qry = """
SELECT name, test_date,
       tot_rom_0to10, tot_rom_10to20, tot_rom_20to30,
       fatigue_index_10, fatigue_index_20, fatigue_index_30,
       total_score, forearm_rom_0to10, forearm_rom_10to20, forearm_rom_20to30
FROM pro_sup_data
"""
# ---------------------------------------------------------------------------
# 1)  CONNECT + LOAD DATA
# ---------------------------------------------------------------------------
with sqlite3.connect(DB_PATH) as conn:
    df = pd.read_sql_query(qry, conn)

df["test_date"] = pd.to_datetime(df["test_date"])

# convert numeric columns
numeric_cols = [
    "tot_rom_0to10","tot_rom_10to20","tot_rom_20to30",
    "forearm_rom_0to10","forearm_rom_10to20","forearm_rom_20to30",
    "fatigue_index_10","fatigue_index_20","fatigue_index_30",
    "total_score"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# ---------------------------------------------------------------------------
# 2)  ADD PERCENTILE COLUMNS  (no merging!)
# ---------------------------------------------------------------------------
metric_cols = ["tot_rom_0to10","tot_rom_10to20","tot_rom_20to30","total_score"]
for col in metric_cols:
    df[f"{col}_pct"] = df[col].rank(pct=True) * 100
    
# ---------------------------------------------------------------------------
# 3)  PICK THE ONE ROW THAT MATCHES THE GLOBALS
# ---------------------------------------------------------------------------
if "NEW_NAME" not in globals() or "NEW_TEST_DATE" not in globals():
    raise RuntimeError(
        "NEW_NAME / NEW_TEST_DATE are not defined in the kernel.\n"
        "Run the ingestion cell first."
    )

target_name  = NEW_NAME
target_date  = pd.to_datetime(NEW_TEST_DATE)

df_target = df[
    (df["name"] == target_name) &
    (df["test_date"] == target_date)
]

if df_target.empty:
    raise RuntimeError(
        f"No row in the DB matches name='{target_name}' "
        f"and test_date='{target_date.date()}'."
    )


# ───────────────────────────────────────────────────────── helper functions ──
ACCENT   = colors.HexColor(ACCENT_HEX)
MUTED    = colors.HexColor(MUTED_HEX)
SHADOW   = colors.HexColor("#0a0d10")

def pct_color(p):
    if p < 33: return "red"
    if p < 67: return "yellow"
    return "limegreen"

def ring_gauge(pct, file_path, size=240):
    pct = max(0,min(100,pct))
    fig = go.Figure(go.Pie(
        values=[pct,100-pct], hole=0.78,
        marker=dict(colors=[pct_color(pct),"#303030"]),
        sort=False, direction="clockwise", textinfo="none"
    ))
    fig.add_annotation(text=f"<b>{int(pct)}%</b>",
                       font=dict(size=size*0.13,color="white"),
                       showarrow=False)
    fig.update_layout(margin=dict(l=0,r=0,t=0,b=0),
                      paper_bgcolor="black", showlegend=False)
    fig.write_image(file_path,width=size,height=size)   # needs kaleido
    return file_path

def rom_bars(rom0, rom1, rom2, highlight_idx, fp, size=(120,120)):
    """3‑bar vertical ROM chart with ~1‑pixel gaps."""
    vals   = [rom0, rom1, rom2]
    colors = [MUTED_HEX]*3
    colors[highlight_idx] = ACCENT_HEX

    fig = go.Figure(go.Bar(
        x=["0", "1", "2"],          # dummy labels (hidden)
        y=vals,
        marker_color=colors,
        width=0.9,                  # bar covers 90% of slot
        hoverinfo="none",
    ))
    fig.update_layout(
        bargap=0.02,                # ≈ 1‑px at 220‑px width
        bargroupgap=0,
        xaxis=dict(visible=False, fixedrange=True),
        yaxis=dict(visible=False, fixedrange=True),
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor="black",
        plot_bgcolor="black",
    )
    fig.write_image(fp, width=size[0], height=size[1])
    return fp



# ──── 2) BUILD PDF ───────────────────────────────────────────────────────────
for _, row in df_target.iterrows():
    athlete = row["name"]; date_str = row["test_date"].date().isoformat()
    pdf_out = os.path.join(REPORT_DIR,
                           f"{athlete} {date_str} Performance Report.pdf")

    c = canvas.Canvas(pdf_out, pagesize=letter)
    W,H  = letter
    tmp  = []                      # collect temp PNGs for cleanup

    # ----- gradient slate background (two rectangles) -----------------------
    c.setFillColor(colors.HexColor("#0e1116"))   # deep slate‑gray
    c.rect(0, 0, W, H, stroke=0, fill=1)

    # ----- vertical teal bar -------------------------------------------------
    c.setFillColor(ACCENT); c.rect(10,0,6,H,stroke=0,fill=1)

    # ----- header ------------------------------------------------------------
    # --- logo without distortion -----------------------------------
    if os.path.exists(LOGO_PATH):
        from reportlab.lib.utils import ImageReader
        img = ImageReader(LOGO_PATH)
        iw, ih = img.getSize()          # image’s natural size in pixels
    
        target_w = 140                  # pick a width that fits
        scale    = target_w / iw
        target_h = ih * scale           # keep the natural aspect
    
        c.drawImage(
            LOGO_PATH,
            W - target_w - 20,          # 20‑pt right margin
            H - target_h - 20,          # 20‑pt top margin
            width  = target_w,
            height = target_h,
            mask   = 'auto'
        )

    c.setFont("Helvetica-BoldOblique",18); c.setFillColor(colors.white)
    c.drawString(30, H-40, f"Name: {athlete}")
    c.drawString(30, H-60, f"Test Date: {date_str}")

    c.setStrokeColor(ACCENT); c.setLineWidth(2)
    c.line(20, H-75, W-20, H-75)

    # ----- card layout helpers ----------------------------------------------
    card_x, card_w = 40, W - 80     # <- was 30 / W-60
    shadow_off, card_radius = 4, 10
    y = H-100

    def draw_card_shadow(y_top, h):
        c.setFillColor(SHADOW)
        c.roundRect(card_x+shadow_off, y_top-h-shadow_off,
                    card_w, h, card_radius, fill=1, stroke=0)

    def draw_card(y_top, h):
        # card body
        c.setFillColor(colors.black); c.setStrokeColor(colors.white)
        c.roundRect(card_x, y_top-h, card_w, h, card_radius, fill=1, stroke=1)


    # ===================== SCORE CARD ========================================
    score_h = 124
    draw_card_shadow(y, score_h)
    draw_card(y, score_h)

    c.setFont("Helvetica-BoldOblique",24); c.setFillColor(ACCENT)
    c.drawString(card_x+140, y-45, "Score")
    c.setFont("Helvetica-BoldOblique",46); c.setFillColor(colors.white)
    c.drawString(card_x+140, y-92, f"{row['total_score']:.1f}")

    png = ring_gauge(row["total_score_pct"], "_score.png", size=230)
    tmp.append(png)
    c.drawImage(ImageReader(png), card_x+card_w-210, y-score_h+8,
                width=108, height=108, mask='auto')

    y -= score_h + 28

    # ===================== INTERVAL CARDS ====================================
    intervals = [
        ("0–10 Seconds", 0, "tot_rom_0to10","fatigue_index_10","forearm_rom_0to10"),
        ("10–20 Seconds",1, "tot_rom_10to20","fatigue_index_20", "forearm_rom_10to20"),
        ("20–30 Seconds",2, "tot_rom_20to30","fatigue_index_30", "forearm_rom_20to30"),
    ]
    for title, idx, rom_col, fat_col, rom_col2 in intervals:
        card_h = 150
        draw_card_shadow(y,card_h)
        draw_card(y,card_h)

        # titles & data
        c.setFont("Helvetica-BoldOblique",18); c.setFillColor(ACCENT)
        c.drawString(card_x+40, y-35, title)

        c.setFont("Helvetica-Oblique",16); c.setFillColor(colors.white)
        c.drawString(card_x+40, y-65,
                     f"Cumulative ROM: {row[rom_col]:.1f}°")
        c.drawString(card_x+40, y-90,
                     f"Fatigue Index:       {row[fat_col]:.1f}%")
        c.drawString(card_x+40, y-115,
                     f"ROM:                     {row[rom_col2]:.1f}°")

        # --- mini bar chart (ROMs) ---
        bars_png = rom_bars(row["tot_rom_0to10"], row["tot_rom_10to20"],
                            row["tot_rom_20to30"], idx,
                            f"_bars_{idx}.png", size=(110,110))
        tmp.append(bars_png)
        c.drawImage(ImageReader(bars_png),
                    card_x+card_w/2-0, y-card_h+25,
                    width=110, height=110, mask='auto')

        # --- ring gauge on right ---
        ring_png = ring_gauge(row[f"{rom_col}_pct"],
                              f"_ring_{idx}.png", size=240)
        tmp.append(ring_png)
        c.drawImage(ImageReader(ring_png), card_x+card_w-110, y-card_h+30,
                    width=90, height=90, mask='auto')

        y -= card_h + 28

    # ------------------------------ finalize page ----------------------------
    c.showPage(); c.save()
    print("✅ Saved", pdf_out)

    for p in tmp:
        if os.path.exists(p): os.remove(p)


✅ Saved D:/Pro-Sup Test/Reports\File, Reese 2025-10-30 Performance Report.pdf


In [ ]:
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import sqlite3
import plotly.express as px
from plotly import graph_objects as go

# Path to your database
db_path = "D:/Pro-Sup Test/pro-sup_data.sqlite"

# Load the data from the database
connection = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM pro_sup_data;", connection)
connection.close()

# Initialize the Dash app with SLATE (dark) Bootstrap theme
app = Dash(__name__, external_stylesheets=[dbc.themes.SLATE])

# Layout for the dashboard
app.layout = dbc.Container([
    # Title Row
    dbc.Row([
        dbc.Col(
            html.H1(
                "Athlete Performance Dashboard",
                className="text-center mb-4"
            ),
            width=12
        )
    ]),

    # Row for Score Gauge (Top & Center)
    dbc.Row([
        dbc.Col(
            dcc.Graph(id="total-score-chart", style={"height": "300px"}),
            # Offset by 3 so it's centered in a 12-column layout (3 + 6 + 3 = 12)
            width={"size": 6, "offset": 3}
        )
    ], className="mb-4"),

    # Row for Athlete Selection
    dbc.Row([
        dbc.Col([
            html.Label("Select Athlete:", style={"color": "black"}),
            dcc.Dropdown(
                id="athlete-dropdown",
                options=[{"label": name, "value": name} for name in df['name'].unique()],
                placeholder="Select an athlete",
                style={
                    "backgroundColor": "slategray",  # forces the dropdown to have white background
                    "color": "black"             # forces the text (and placeholder text) to be black
                }
            )
        ], width=4)
    ], className="mb-4"),

    # Row for Printed-Out Column Values
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader("Data Overview for Selected Athlete"),
                dbc.CardBody([
                    html.Div(id="column-values")  # We'll fill this with text in the callback
                ])
            ])
        ], width=12)
    ], className="mb-4"),

    # Row for Performance Chart (ROM)
    dbc.Row([
        dbc.Col(dcc.Graph(id="performance-chart"), width=12)
    ], className="mb-4"),

    # Row for Fatigue Chart (left) and (optionally reuse gauge or another chart on right)
    dbc.Row([
        dbc.Col(dcc.Graph(id="fatigue-chart"), width=6),
        dbc.Col([], width=6)  # Placeholder if you want another chart or remove if not needed
    ], className="mb-4")

], fluid=True)

@app.callback(
    Output("performance-chart", "figure"),
    Output("fatigue-chart", "figure"),
    Output("total-score-chart", "figure"),
    Output("column-values", "children"),
    Input("athlete-dropdown", "value")
)
def update_charts(selected_athlete):
    # If nothing is selected, return empty figures and no text
    if not selected_athlete:
        return {}, {}, {}, ""

    # Filter the DataFrame for the selected athlete
    athlete_data = df[df['name'] == selected_athlete]

    # 1) Performance Chart (ROM) over intervals
    performance_data = {
        "Interval": ["0-10 sec", "10-20 sec", "20-30 sec"],
        "ROM": [
            athlete_data.iloc[0]['tot_rom_0to10'],
            athlete_data.iloc[0]['tot_rom_10to20'],
            athlete_data.iloc[0]['tot_rom_20to30']
        ]
    }
    performance_fig = px.bar(
        pd.DataFrame(performance_data),
        x="Interval",
        y="ROM",
        color_discrete_sequence=["SteelBlue"],
        labels={"Interval": "Time Interval", "ROM": "Range of Motion (ROM)"},
        title="ROM Across Intervals",
        text="ROM"  # Display bar values
    )
    performance_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    performance_fig.update_layout(yaxis=dict(range=[0, max(performance_data["ROM"])*1.2]))

    # 2) Fatigue Chart
    fatigue_data = {
        "Interval": ["0-10 sec", "10-20 sec", "20-30 sec"],
        "Fatigue Index": [
            athlete_data.iloc[0]['fatigue_index_10'],
            athlete_data.iloc[0]['fatigue_index_20'],
            athlete_data.iloc[0]['fatigue_index_30']
        ]
    }
    fatigue_fig = px.bar(
        pd.DataFrame(fatigue_data),
        x="Interval",
        y="Fatigue Index",
        color_discrete_sequence=["SteelBlue"],
        labels={"Interval": "Time Interval", "Fatigue Index": "Fatigue Index (%)"},
        title="Fatigue Indices",
        text="Fatigue Index"
    )
    fatigue_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
    fatigue_fig.update_layout(yaxis=dict(range=[0, max(fatigue_data["Fatigue Index"])*1.2]))

    # 3) Total Score Chart (Gauge)
    total_score_value = athlete_data.iloc[0]['total_score']
    total_score_fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=total_score_value,
        title={"text": "Total Score"},
        gauge={
            "axis": {"range": [0, 100]},
            "bar": {"color": "steelblue"}
        }
    ))
    total_score_fig.update_layout(margin=dict(l=40, r=40, t=50, b=40))

    # 4) Column Values for Easy Reading
    #    Add as many columns as you want to display from your DataFrame
    rom_0_10 = athlete_data.iloc[0]['tot_rom_0to10']
    rom_10_20 = athlete_data.iloc[0]['tot_rom_10to20']
    rom_20_30 = athlete_data.iloc[0]['tot_rom_20to30']
    fatigue_10 = athlete_data.iloc[0]['fatigue_index_10']
    fatigue_20 = athlete_data.iloc[0]['fatigue_index_20']
    fatigue_30 = athlete_data.iloc[0]['fatigue_index_30']

    # Create a set of paragraphs or list items to display
    column_info = [
        html.P(f"ROM 0-10 sec: {rom_0_10:.2f}"),
        html.P(f"ROM 10-20 sec: {rom_10_20:.2f}"),
        html.P(f"ROM 20-30 sec: {rom_20_30:.2f}"),
        html.P(f"Fatigue Index 0-10 sec: {fatigue_10:.2f}%"),
        html.P(f"Fatigue Index 10-20 sec: {fatigue_20:.2f}%"),
        html.P(f"Fatigue Index 20-30 sec: {fatigue_30:.2f}%"),
        html.P(f"Total Score: {total_score_value:.2f}")
    ]
    
    return performance_fig, fatigue_fig, total_score_fig, column_info


# Run the Dash app
if __name__ == "__main__":
    app.run_server(debug=True)


In [ ]:
# from dash import Dash, dcc, html, Input, Output
# import dash_bootstrap_components as dbc
# import pandas as pd
# import sqlite3
# import plotly.express as px
# from plotly import graph_objects as go
# 
# # Path to your database
# db_path = "D:/Pro-Sup Test/pro-sup_data.sqlite"
# 
# # Load the data from the database
# connection = sqlite3.connect(db_path)
# df = pd.read_sql_query("SELECT * FROM pro_sup_data;", connection)
# connection.close()
# 
# # Initialize the Dash app with Bootstrap CSS
# app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
# 
# # Layout for the dashboard
# app.layout = dbc.Container([
#     dbc.Row([
#         dbc.Col(html.H1("Athlete Performance Dashboard", className="text-center mb-4"), width=12)
#     ]),
#     dbc.Row([
#         dbc.Col([
#             html.Label("Select Athlete:"),
#             dcc.Dropdown(
#                 id="athlete-dropdown",
#                 options=[{"label": name, "value": name} for name in df['name'].unique()],
#                 placeholder="Select an athlete"
#             )
#         ], width=4),
#     ]),
#     dbc.Row([
#         dbc.Col(dcc.Graph(id="performance-chart"), width=12)
#     ]),
#     dbc.Row([
#         dbc.Col(dcc.Graph(id="fatigue-chart"), width=6),
#         dbc.Col(dcc.Graph(id="total-score-chart"), width=6),
#     ])
# ], fluid=True)
# 
# @app.callback(
#     [Output("performance-chart", "figure"),
#      Output("fatigue-chart", "figure"),
#      Output("total-score-chart", "figure")],
#     [Input("athlete-dropdown", "value")]
# )
# def update_charts(selected_athlete):
#     if not selected_athlete:
#         return {}, {}, {}
# 
#     athlete_data = df[df['name'] == selected_athlete]
# 
#     # Performance Chart: ROM over intervals
#     performance_data = {
#         "Interval": ["0-10 sec", "10-20 sec", "20-30 sec"],
#         "ROM": [
#             athlete_data.iloc[0]['tot_rom_0to10'],
#             athlete_data.iloc[0]['tot_rom_10to20'],
#             athlete_data.iloc[0]['tot_rom_20to30']
#         ]
#     }
#     performance_fig = px.bar(
#         pd.DataFrame(performance_data),
#         x="Interval",
#         y="ROM",
#         labels={"Interval": "Time Interval", "ROM": "Range of Motion (ROM)"},
#         title="ROM Across Intervals",
#         text="ROM"  # Add values as text
#     )
#     performance_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # Format and position text
# 
#     # Fatigue Chart: Fatigue Indices
#     fatigue_data = {
#         "Interval": ["0-10 sec", "10-20 sec", "20-30 sec"],
#         "Fatigue Index": [
#             athlete_data.iloc[0]['fatigue_index_10'],
#             athlete_data.iloc[0]['fatigue_index_20'],
#             athlete_data.iloc[0]['fatigue_index_30']
#         ]
#     }
#     fatigue_fig = px.bar(
#         pd.DataFrame(fatigue_data),
#         x="Interval",
#         y="Fatigue Index",
#         labels={"Interval": "Time Interval", "Fatigue Index": "Fatigue Index (%)"},
#         title="Fatigue Indices",
#         text="Fatigue Index"  # Add values as text
#     )
#     fatigue_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')  # Format and position text
# 
#     # Total Score Chart: Single Value
#     total_score_fig = go.Figure(go.Indicator(
#         mode="gauge+number",
#         value=athlete_data.iloc[0]['total_score'],
#         title={"text": "Total Score"},
#         gauge={
#             "axis": {"range": [0, 100]},
#             "bar": {"color": "darkblue"}
#         }
#     ))
# 
#     return performance_fig, fatigue_fig, total_score_fig
# 
# 
# # Run the Dash app
# if __name__ == "__main__":
#     app.run_server(debug=True)


In [ ]:
# Reorders the database to be in alphabetical order

import sqlite3


db_path = "D:/Pro-Sup Test/pro-sup_data.sqlite" 
sort_column = "name"     

def reorder_all_tables(db_path, sort_column):
    try:
        # Connect to the database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Fetch all table names in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]

            # Skip system tables like sqlite_sequence
            if table_name.startswith("sqlite_"):
                continue

            print(f"Processing table: {table_name}")

            # Check if the column exists in the current table
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = [info[1] for info in cursor.fetchall()]
            if sort_column not in columns:
                print(f"Skipping table '{table_name}' - Column '{sort_column}' not found.")
                continue

            # Create a new sorted table
            temp_table = f"{table_name}_sorted"
            cursor.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM {table_name} ORDER BY {sort_column} ASC;")
            
            # Drop the old table
            cursor.execute(f"DROP TABLE {table_name};")
            
            # Rename the new table to the original name
            cursor.execute(f"ALTER TABLE {temp_table} RENAME TO {table_name};")
            print(f"Table '{table_name}' reordered successfully.")

        # Commit changes
        conn.commit()
        print("All tables processed.")
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    finally:
        conn.close()

reorder_all_tables(db_path, sort_column)
